In [15]:
import os
import json
import re
import requests
import datetime
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process

import google.generativeai as genai
from langchain_community.chat_models import ChatLiteLLM
from langchain.tools import BaseTool, Tool
from pydantic import BaseModel, Field
from bs4 import BeautifulSoup

# Load environment variables
load_dotenv()

# API Keys
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# Verify API keys are loaded
if not SERPER_API_KEY or not GEMINI_API_KEY:
    print("Error: Required API keys are missing. Please check your .env file.")
    exit(1)

print(f"SERPER_API_KEY loaded: {'Yes' if SERPER_API_KEY else 'No'}")
print(f"GEMINI_API_KEY loaded: {'Yes' if GEMINI_API_KEY else 'No'}")

# Define tools for agents
class SearchTool(BaseTool):
    name: str = "Search Tool"
    description: str = "Searches for information on the web using Google Search API"
    
    def _run(self, query: str) -> str:
        try:
            url = "https://google.serper.dev/search"
            payload = json.dumps({"q": query})
            headers = {
                'X-API-KEY': SERPER_API_KEY,
                'Content-Type': 'application/json'
            }
            response = requests.request("POST", url, headers=headers, data=payload)
            return response.text
        except Exception as e:
            return f"Error during search: {str(e)}"

class EventScraper(BaseTool):
    name: str = "Event Scraper"
    description: str = "Scrapes event details from provided URLs"
    
    def _run(self, url: str) -> str:
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'
            }
            response = requests.get(url, headers=headers)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Extract page content as text
            page_text = soup.get_text()
            
            # Return filtered page content with most relevant info
            return page_text[:10000]  # Limit size for processing
        except Exception as e:
            return f"Error scraping event details: {str(e)}"

class LinkedInScraper(BaseTool):
    name: str = "LinkedIn Scraper"
    description: str = "Searches LinkedIn for events, jobs, and opportunities"
    
    def _run(self, query: str) -> str:
        try:
            # Use SERP API for LinkedIn search
            url = "https://google.serper.dev/search"
            enhanced_query = f"site:linkedin.com/events OR site:linkedin.com/jobs {query}"
            payload = json.dumps({"q": enhanced_query})
            headers = {
                'X-API-KEY': SERPER_API_KEY,
                'Content-Type': 'application/json'
            }
            response = requests.request("POST", url, headers=headers, data=payload)
            return response.text
        except Exception as e:
            return f"Error during LinkedIn search: {str(e)}"

class InternshalaFinder(BaseTool):
    name: str = "Internshala Finder"
    description: str = "Searches Internshala for internships and training programs"
    
    def _run(self, query: str) -> str:
        try:
            url = "https://google.serper.dev/search"
            enhanced_query = f"site:internshala.com {query} internship OR training"
            payload = json.dumps({"q": enhanced_query})
            headers = {
                'X-API-KEY': SERPER_API_KEY,
                'Content-Type': 'application/json'
            }
            response = requests.request("POST", url, headers=headers, data=payload)
            return response.text
        except Exception as e:
            return f"Error during Internshala search: {str(e)}"

class EventbriteAndMeetupFinder(BaseTool):
    name: str = "Eventbrite and Meetup Finder"
    description: str = "Searches Eventbrite and Meetup for tech events and workshops"
    
    def _run(self, query: str) -> str:
        try:
            url = "https://google.serper.dev/search"
            enhanced_query = f"site:eventbrite.com OR site:meetup.com {query} event OR workshop OR hackathon"
            payload = json.dumps({"q": enhanced_query})
            headers = {
                'X-API-KEY': SERPER_API_KEY,
                'Content-Type': 'application/json'
            }
            response = requests.request("POST", url, headers=headers, data=payload)
            return response.text
        except Exception as e:
            return f"Error during Eventbrite/Meetup search: {str(e)}"

class UniversityAndTechCommunityFinder(BaseTool):
    name: str = "University and Tech Community Finder"
    description: str = "Searches university pages and tech communities for events"
    
    def _run(self, query: str) -> str:
        try:
            url = "https://google.serper.dev/search"
            enhanced_query = f"{query} university OR tech community OR bootcamp OR workshop"
            payload = json.dumps({"q": enhanced_query})
            headers = {
                'X-API-KEY': SERPER_API_KEY,
                'Content-Type': 'application/json'
            }
            response = requests.request("POST", url, headers=headers, data=payload)
            return response.text
        except Exception as e:
            return f"Error during university/community search: {str(e)}"

class CompanyCareerScraper(BaseTool):
    name: str = "Company Career Page Scraper"
    description: str = "Searches company career pages for job opportunities and events"
    
    def _run(self, query: str) -> str:
        try:
            url = "https://google.serper.dev/search"
            enhanced_query = f"{query} company career OR job OR hiring OR recruiting event"
            payload = json.dumps({"q": enhanced_query})
            headers = {
                'X-API-KEY': SERPER_API_KEY,
                'Content-Type': 'application/json'
            }
            response = requests.request("POST", url, headers=headers, data=payload)
            return response.text
        except Exception as e:
            return f"Error during company career search: {str(e)}"

# Helper functions that implement the tool functionality
def search_function(query: str) -> str:
    try:
        url = "https://google.serper.dev/search"
        payload = json.dumps({"q": query})
        headers = {
            'X-API-KEY': SERPER_API_KEY,
            'Content-Type': 'application/json'
        }
        response = requests.request("POST", url, headers=headers, data=payload)
        return response.text
    except Exception as e:
        return f"Error during search: {str(e)}"

def linkedin_search_function(query: str) -> str:
    try:
        url = "https://google.serper.dev/search"
        enhanced_query = f"site:linkedin.com/events OR site:linkedin.com/jobs {query}"
        payload = json.dumps({"q": enhanced_query})
        headers = {
            'X-API-KEY': SERPER_API_KEY,
            'Content-Type': 'application/json'
        }
        response = requests.request("POST", url, headers=headers, data=payload)
        return response.text
    except Exception as e:
        return f"Error during LinkedIn search: {str(e)}"

def internshala_search_function(query: str) -> str:
    try:
        url = "https://google.serper.dev/search"
        enhanced_query = f"site:internshala.com {query} internship OR training"
        payload = json.dumps({"q": enhanced_query})
        headers = {
            'X-API-KEY': SERPER_API_KEY,
            'Content-Type': 'application/json'
        }
        response = requests.request("POST", url, headers=headers, data=payload)
        return response.text
    except Exception as e:
        return f"Error during Internshala search: {str(e)}"

def eventbrite_meetup_search_function(query: str) -> str:
    try:
        url = "https://google.serper.dev/search"
        enhanced_query = f"site:eventbrite.com OR site:meetup.com {query} event OR workshop OR hackathon"
        payload = json.dumps({"q": enhanced_query})
        headers = {
            'X-API-KEY': SERPER_API_KEY,
            'Content-Type': 'application/json'
        }
        response = requests.request("POST", url, headers=headers, data=payload)
        return response.text
    except Exception as e:
        return f"Error during Eventbrite/Meetup search: {str(e)}"

def university_tech_search_function(query: str) -> str:
    try:
        url = "https://google.serper.dev/search"
        enhanced_query = f"{query} university OR tech community OR bootcamp OR workshop"
        payload = json.dumps({"q": enhanced_query})
        headers = {
            'X-API-KEY': SERPER_API_KEY,
            'Content-Type': 'application/json'
        }
        response = requests.request("POST", url, headers=headers, data=payload)
        return response.text
    except Exception as e:
        return f"Error during university/community search: {str(e)}"

def company_career_search_function(query: str) -> str:
    try:
        url = "https://google.serper.dev/search"
        enhanced_query = f"{query} company career OR job OR hiring OR recruiting event"
        payload = json.dumps({"q": enhanced_query})
        headers = {
            'X-API-KEY': SERPER_API_KEY,
            'Content-Type': 'application/json'
        }
        response = requests.request("POST", url, headers=headers, data=payload)
        return response.text
    except Exception as e:
        return f"Error during company career search: {str(e)}"

def event_scraper_function(url: str) -> str:
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract page content as text
        page_text = soup.get_text()
        
        # Return filtered page content with most relevant info
        return page_text[:10000]  # Limit size for processing
    except Exception as e:
        return f"Error scraping event details: {str(e)}"
    
# Define Agents
def create_profile_analyzer_agent():
    llm = ChatLiteLLM(
        model="gemini/gemini-1.5-flash",
        api_key=GEMINI_API_KEY,
        temperature=0.2
    )
    
    return Agent(
        role="User Profile Analyzer",
        goal="Extract key profile data and generate targeted search queries",
        backstory="""You're an expert data analyst who specializes in understanding 
        user profiles and extracting the most relevant information to match them with 
        opportunities. You know how to identify skills, experience levels, and career 
        aspirations to build effective search queries.""",
        verbose=True,
        allow_delegation=False,
        llm=llm
    )
def create_opportunity_scout_agent():
    llm = ChatLiteLLM(
        model="gemini/gemini-1.5-flash",
        api_key=GEMINI_API_KEY,
        temperature=0.2
    )
    
    # Create proper Tool instances
    search_tool = Tool.from_function(
        func=search_function,
        name="Search Tool",
        description="Searches for information on the web using Google Search API"
    )
    
    linkedin_scraper = Tool.from_function(
        func=linkedin_search_function,
        name="LinkedIn Scraper",
        description="Searches LinkedIn for events, jobs, and opportunities"
    )
    
    internshala_finder = Tool.from_function(
        func=internshala_search_function,
        name="Internshala Finder",
        description="Searches Internshala for internships and training programs"
    )
    
    eventbrite_meetup_finder = Tool.from_function(
        func=eventbrite_meetup_search_function,
        name="Eventbrite and Meetup Finder",
        description="Searches Eventbrite and Meetup for tech events and workshops"
    )
    
    university_tech_finder = Tool.from_function(
        func=university_tech_search_function,
        name="University and Tech Community Finder",
        description="Searches university pages and tech communities for events"
    )
    
    company_career_scraper = Tool.from_function(
        func=company_career_search_function,
        name="Company Career Page Scraper",
        description="Searches company career pages for job opportunities and events"
    )
    
    event_scraper = Tool.from_function(
        func=event_scraper_function,
        name="Event Scraper",
        description="Scrapes event details from provided URLs"
    )
    
    return Agent(
        role="Opportunity Scout",
        goal="Find high-quality events and opportunities matching user's profile",
        backstory="""You're an expert researcher who specializes in finding opportunities 
        that help people advance their careers. You know how to navigate various websites, 
        extract relevant information, and identify truly valuable opportunities. You're 
        skilled at filtering out misleading or low-quality events.""",
        verbose=True,
        allow_delegation=True,
        tools=[
            search_tool,
            linkedin_scraper,
            internshala_finder,
            eventbrite_meetup_finder,
            university_tech_finder,
            company_career_scraper,
            event_scraper
        ],
        llm=llm
    )
def create_event_evaluator_agent():
    llm = ChatLiteLLM(
        model="gemini/gemini-1.5-flash",
        api_key=GEMINI_API_KEY,
        temperature=0.2
    )
    
    return Agent(
        role="Event Evaluator",
        goal="Rank and filter events based on relevance to user profile",
        backstory="""You're an analytical expert who specializes in evaluating the 
        quality and relevance of opportunities. You know how to assess events based on 
        their alignment with a user's skills, goals, and preferences. You can filter out 
        misleading or irrelevant opportunities and rank the best ones.""",
        verbose=True,
        allow_delegation=False,
        llm=llm
    )

def create_recommender_agent():
    llm = ChatLiteLLM(
        model="gemini/gemini-1.5-flash",
        api_key=GEMINI_API_KEY,
        temperature=0.2
    )
    
    return Agent(
        role="Opportunity Recommender",
        goal="Create personalized recommendations with detailed reasoning",
        backstory="""You're a career advisor who specializes in matching people with the 
        perfect opportunities to advance their careers. You know how to explain why an 
        opportunity is valuable for someone's specific situation and how it aligns with 
        their goals and skills. You provide detailed, actionable information.""",
        verbose=True,
        allow_delegation=False,
        llm=llm
    )

# Define Tasks
def create_profile_analysis_task(profile_analyzer_agent, user_profile):
    return Task(
        description=f"""
        Analyze the user profile and extract key information:
        
        User Profile:
        {json.dumps(user_profile, indent=2)}
        
        Your task:
        1. Extract and categorize the user's skills (technical, soft, domain-specific)
        2. Identify their educational background and level
        3. Determine their career stage (student, early career, experienced)
        4. Understand their short and long-term career goals
        5. Note their location preferences and mobility
        6. Identify their interest in mentorship and community involvement
        
        Based on this analysis, generate:
        1. A summary of the user's profile
        2. A list of 5-7 search queries that would be most effective for finding relevant opportunities
        3. A list of keywords to use for filtering results
        
        Return the analysis as a structured JSON object with the following format:
        {
            "profile_summary": "Brief profile overview",
            "search_queries": ["query1", "query2", ...],
            "filter_keywords": ["keyword1", "keyword2", ...],
            "career_stage": "student/early/mid/senior",
            "location_preference": "remote/local/flexible",
            "primary_skills": ["skill1", "skill2", ...],
            "learning_goals": ["goal1", "goal2", ...]
        }
        """,
        agent=profile_analyzer_agent,
        expected_output="Structured JSON with profile analysis and search queries"
    )

def create_opportunity_search_task(opportunity_scout_agent, profile_analysis):
    return Task(
        description=f"""
        Using the profile analysis provided, search for high-quality opportunities across various platforms:
        
        Profile Analysis:
        {json.dumps(profile_analysis, indent=2)}
        
        Your task:
        1. Use the search queries and keywords to search across:
           - LinkedIn (events, jobs, learning opportunities)
           - Internshala (internships, training programs)
           - Eventbrite & Meetup (workshops, hackathons, conferences)
           - University pages & Tech communities (webinars, bootcamps)
           - Company career pages (job fairs, recruitment events)
        
        2. For each potential opportunity, collect:
           - Event/Opportunity name
           - Description
           - Skills covered
           - Location (online or physical)
           - Date & Time
           - Registration link or application process
           - Source website
        
        3. Find at least 10-15 different opportunities to ensure variety
        
        4. Focus on opportunities that match the user's:
           - Primary skills
           - Career stage
           - Learning goals
           - Location preferences
        
        Return the collected opportunities as a structured JSON array with the following format:
        [
            {
                "title": "Event title",
                "description": "Event description",
                "skills_covered": ["skill1", "skill2", ...],
                "location": "Online/City name",
                "date_time": "Date and time",
                "registration_link": "URL",
                "source": "Website name",
                "type": "workshop/hackathon/internship/etc"
            },
            ...
        ]
        """,
        agent=opportunity_scout_agent,
        expected_output="Structured JSON array with 10-15 opportunities"
    )

def create_event_evaluation_task(event_evaluator_agent, user_profile, opportunities):
    return Task(
        description=f"""
        Evaluate and rank the opportunities based on relevance to the user's profile:
        
        User Profile:
        {json.dumps(user_profile, indent=2)}
        
        Opportunities:
        {json.dumps(opportunities, indent=2)}
        
        Your task:
        1. Evaluate each opportunity based on:
           - Relevance to user's skills (immediate skill match)
           - Alignment with career goals (short and long term)
           - Quality of the opportunity (reputation, content depth)
           - Timeliness (upcoming deadlines, registration periods)
           - Value for skill development
        
        2. Score each opportunity on a scale of 1-10 for each criterion
        
        3. Filter out any opportunities that:
           - Are clearly irrelevant to the user's profile
           - Have already passed (if date information is available)
           - Appear to be misleading or low quality
           - Don't match the user's location preferences
        
        4. Rank the remaining opportunities by overall score
        
        Return the top 8-10 evaluated opportunities as a structured JSON array with the following format:
        [
            {
                "title": "Event title",
                "description": "Event description",
                "skills_covered": ["skill1", "skill2", ...],
                "location": "Online/City name",
                "date_time": "Date and time",
                "registration_link": "URL",
                "source": "Website name",
                "type": "workshop/hackathon/internship/etc",
                "relevance_score": 8.5,
                "goal_alignment_score": 7.0,
                "quality_score": 9.0,
                "timeliness_score": 8.0,
                "overall_score": 8.1,
                "relevance_explanation": "Brief explanation of relevance"
            },
            ...
        ]
        """,
        agent=event_evaluator_agent,
        expected_output="Structured JSON array with evaluated opportunities"
    )

def create_recommendation_task(recommender_agent, user_profile, evaluated_opportunities):
    return Task(
        description=f"""
        Create personalized recommendations for the top 5 opportunities:
        
        User Profile:
        {json.dumps(user_profile, indent=2)}
        
        Evaluated Opportunities:
        {json.dumps(evaluated_opportunities, indent=2)}
        
        Your task:
        1. Select the top 5 opportunities with the highest overall scores
        
        2. For each recommended opportunity, create a personalized recommendation that includes:
           - Title and brief description
           - Why it's specifically relevant to this user (based on their profile)
           - Key skills or knowledge they would gain
           - How it aligns with their stated career goals
           - Any preparation or prerequisites needed
           - Clear instructions on how to apply or register
           - Any deadlines or important dates
        
        3. Format the recommendations in a clear, engaging manner
        
        4. Include a brief general recommendation at the end about types of opportunities 
           the user should continue looking for
        
        Return the final recommendations as a structured JSON object with the following format:
        {{
            "top_recommendations": [
                {{
                    "title": "Event title",
                    "description": "Event description",
                    "relevance_to_user": "Detailed explanation of relevance",
                    "skills_to_gain": ["skill1", "skill2", ...],
                    "goal_alignment": "How this aligns with career goals",
                    "preparation_needed": "Any prerequisites",
                    "how_to_apply": "Registration process",
                    "important_dates": "Deadlines",
                    "registration_link": "URL",
                    "source": "Website name"
                }},
                ...
            ],
            "general_advice": "General recommendation for future opportunities"
        }}
        """,
        agent=recommender_agent,
        expected_output="Structured JSON with 5 detailed recommendations"
    )

# Main function to run the opportunity scout system
def find_opportunities(user_profile):
    print("Starting Opportunity Scout AI...")
    
    # Create agents
    print("Creating specialized agents...")
    profile_analyzer_agent = create_profile_analyzer_agent()
    opportunity_scout_agent = create_opportunity_scout_agent()
    event_evaluator_agent = create_event_evaluator_agent()
    recommender_agent = create_recommender_agent()
    
    # Create tasks
    print("Creating analysis task...")
    profile_analysis_task = create_profile_analysis_task(profile_analyzer_agent, user_profile)
    
    # Run profile analysis
    profile_analysis_crew = Crew(
        agents=[profile_analyzer_agent],
        tasks=[profile_analysis_task],
        verbose=True,
        process=Process.sequential
    )
    
    print("Analyzing user profile...")
    profile_analysis_result = profile_analysis_crew.kickoff()
    
    try:
        profile_analysis = json.loads(str(profile_analysis_result))
        print("Profile analysis complete!")
    except json.JSONDecodeError:
        # Try to extract JSON from text
        try:
            json_match = re.search(r'```json\n(.*?)\n```', str(profile_analysis_result), re.DOTALL)
            if json_match:
                profile_analysis = json.loads(json_match.group(1))
            else:
                print("Warning: Could not parse profile analysis as JSON. Using raw text.")
                profile_analysis = {"raw_result": str(profile_analysis_result)}
        except Exception as e:
            print(f"Error parsing profile analysis: {str(e)}")
            profile_analysis = {"raw_result": str(profile_analysis_result)}
    
    # Create and run opportunity search task
    print("Creating opportunity search task...")
    opportunity_search_task = create_opportunity_search_task(opportunity_scout_agent, profile_analysis)
    
    opportunity_search_crew = Crew(
        agents=[opportunity_scout_agent],
        tasks=[opportunity_search_task],
        verbose=True,
        process=Process.sequential
    )
    
    print("Searching for opportunities...")
    opportunity_search_result = opportunity_search_crew.kickoff()
    
    try:
        opportunities = json.loads(str(opportunity_search_result))
        print("Opportunity search complete!")
    except json.JSONDecodeError:
        # Try to extract JSON from text
        try:
            json_match = re.search(r'```json\n(.*?)\n```', str(opportunity_search_result), re.DOTALL)
            if json_match:
                opportunities = json.loads(json_match.group(1))
            else:
                print("Warning: Could not parse opportunities as JSON. Using raw text.")
                opportunities = {"raw_result": str(opportunity_search_result)}
        except Exception as e:
            print(f"Error parsing opportunities: {str(e)}")
            opportunities = {"raw_result": str(opportunity_search_result)}
    
    # Create and run event evaluation task
    print("Creating event evaluation task...")
    event_evaluation_task = create_event_evaluation_task(event_evaluator_agent, user_profile, opportunities)
    
    event_evaluation_crew = Crew(
        agents=[event_evaluator_agent],
        tasks=[event_evaluation_task],
        verbose=True,
        process=Process.sequential
    )
    
    print("Evaluating opportunities...")
    event_evaluation_result = event_evaluation_crew.kickoff()
    
    try:
        evaluated_opportunities = json.loads(str(event_evaluation_result))
        print("Event evaluation complete!")
    except json.JSONDecodeError:
        # Try to extract JSON from text
        try:
            json_match = re.search(r'```json\n(.*?)\n```', str(event_evaluation_result), re.DOTALL)
            if json_match:
                evaluated_opportunities = json.loads(json_match.group(1))
            else:
                print("Warning: Could not parse evaluated opportunities as JSON. Using raw text.")
                evaluated_opportunities = {"raw_result": str(event_evaluation_result)}
        except Exception as e:
            print(f"Error parsing evaluated opportunities: {str(e)}")
            evaluated_opportunities = {"raw_result": str(event_evaluation_result)}
    
    # Create and run recommendation task
    print("Creating recommendation task...")
    recommendation_task = create_recommendation_task(recommender_agent, user_profile, evaluated_opportunities)
    
    recommendation_crew = Crew(
        agents=[recommender_agent],
        tasks=[recommendation_task],
        verbose=True,
        process=Process.sequential
    )
    
    print("Generating final recommendations...")
    recommendation_result = recommendation_crew.kickoff()
    
    try:
        final_recommendations = json.loads(str(recommendation_result))
        print("Recommendations complete!")
    except json.JSONDecodeError:
        # Try to extract JSON from text
        try:
            json_match = re.search(r'```json\n(.*?)\n```', str(recommendation_result), re.DOTALL)
            if json_match:
                final_recommendations = json.loads(json_match.group(1))
            else:
                print("Warning: Could not parse recommendations as JSON. Using raw text.")
                final_recommendations = {"raw_result": str(recommendation_result)}
        except Exception as e:
            print(f"Error parsing recommendations: {str(e)}")
            final_recommendations = {"raw_result": str(recommendation_result)}
    
    print("Opportunity Scout AI process complete!")
    return final_recommendations

# Example usage
if __name__ == "__main__":
    # Example user profile from the problem statement
    user_profile = {
        "education": "Bachelor's Degree",
        "skills": ["Python", "Java", "AI", "NLP", "ML", "DL", "web development", "app development"],
        "current_status": "Looking for Work",
        "experience_years": 1,
        "last_job": {
            "title": "AI developer",
            "company": "OLVT"
        },
        "life_stage": {
            "pregnancy_status": "No",
            "needs_flexible_work": False,
            "situation": "None of the above"
        },
        "job_preferences": {
            "type": "Remote Work",
            "roles": ["Software"],
            "short_term_goal": "Upskill and crack good placement",
            "long_term_goal": "Yes, i want to be an entrepreneur"
        },
        "location": {
            "city": "Tirupati",
            "relocation": True,
            "work_mode": "Flexible"
        },
        "community": {
            "wants_mentorship": True,
            "mentorship_type": "Skill development",
            "join_events": True
        },
        "communication_preference": "Email",
        "consent": True
    }
    
    # Find opportunities for the user
    recommendations = find_opportunities(user_profile)
    
    # Print the recommendations
    print("\n=== TOP RECOMMENDATIONS ===\n")
    for i, rec in enumerate(recommendations.get("top_recommendations", []), 1):
        print(f"{i}. {rec.get('title')}")
        print(f"   Relevance: {rec.get('relevance_to_user')[:100]}...")
        print(f"   Skills: {', '.join(rec.get('skills_to_gain', []))}")
        print(f"   Link: {rec.get('registration_link')}")
        print()
    
    print(f"General Advice: {recommendations.get('general_advice')}")

SERPER_API_KEY loaded: Yes
GEMINI_API_KEY loaded: Yes
Starting Opportunity Scout AI...
Creating specialized agents...


ValidationError: 7 validation errors for Agent
tools.0
  Input should be a valid dictionary or instance of BaseTool [type=model_type, input_value=Tool(name='Search Tool', ... at 0x000002858E956160>), input_type=Tool]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type
tools.1
  Input should be a valid dictionary or instance of BaseTool [type=model_type, input_value=Tool(name='LinkedIn Scrap... at 0x000002858E9565C0>), input_type=Tool]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type
tools.2
  Input should be a valid dictionary or instance of BaseTool [type=model_type, input_value=Tool(name='Internshala Fi... at 0x000002858E955580>), input_type=Tool]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type
tools.3
  Input should be a valid dictionary or instance of BaseTool [type=model_type, input_value=Tool(name='Eventbrite and... at 0x000002858E956660>), input_type=Tool]
    For further information visit

In [17]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
import google.generativeai as genai
import requests
import json
from datetime import datetime

from langchain_community.chat_models import ChatLiteLLM
import re
from pymongo import MongoClient

# Load environment variables
load_dotenv()

# API Keys
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
MONGODB_URI = os.getenv("MONGODB_URI")

# Configure Gemini
genai.configure(api_key=GEMINI_API_KEY)

class EventRecommenderSystem:
    def __init__(self, user_id=None, user_profile=None):
        """Initialize the Event Recommender System"""
        self.user_id = user_id
        self.user_profile = user_profile
        self.search_analyst = self._create_search_analyst()
        self.event_curator = self._create_event_curator()
        self.recommendation_expert = self._create_recommendation_expert()
        self.crew = self._create_crew()
        

    def _create_search_analyst(self):
        """Create the Search Analyst agent"""
        llm = ChatLiteLLM(
            model="gemini/gemini-1.5-flash",
            api_key=GEMINI_API_KEY,
            temperature=0.2
        )
        return Agent(
            role="Search Analyst",
            goal="Find relevant events from multiple sources based on user profile",
            backstory="""You are an expert at crafting search queries and extracting 
            information from the web. You specialize in finding events, 
            workshops, hackathons, and learning opportunities.""",
            verbose=True,
            allow_delegation=True,
            llm=llm,
            tools=[self.search_web]
        )
        
    def _create_event_curator(self):
        """Create the Event Curator agent"""
        llm = ChatLiteLLM(
            model="gemini/gemini-1.5-flash",
            api_key=GEMINI_API_KEY,
            temperature=0.2
        )
        return Agent(
            role="Event Curator",
            goal="Analyze and filter events for relevance to the user profile",
            backstory="""You are an experienced event curator who knows how to 
            match events to a person's skillset, experience, and career goals. 
            You understand what makes an event valuable for professional growth.""",
            verbose=True,
            allow_delegation=True,
            llm=llm
        )
        
    def _create_recommendation_expert(self):
        """Create the Recommendation Expert agent"""
        llm = ChatLiteLLM(
            model="gemini/gemini-1.5-flash",
            api_key=GEMINI_API_KEY,
            temperature=0.2
        )
        return Agent(
            role="Recommendation Expert",
            goal="Create personalized event recommendations with rationale",
            backstory="""You specialize in explaining why certain events are 
            perfect for a person's career growth. You can clearly articulate 
            the benefits and relevance of each recommendation.""",
            verbose=True,
            allow_delegation=False,
            llm=llm
        )
        
    def _create_crew(self):
        """Create the AI Crew"""
        return Crew(
            agents=[self.search_analyst, self.event_curator, self.recommendation_expert],
            tasks=self._create_tasks(),
            verbose=2,
            process=Process.sequential
        )
        
    def _create_tasks(self):
        """Create the sequence of tasks for the crew"""
        task1 = Task(
            description="""
            Analyze the user profile to identify key search parameters:
            1. Primary and secondary skills
            2. Career interests and goals
            3. Educational background
            4. Experience level
            5. Location preferences
            
            Then, search for relevant events from multiple sources including:
            - LinkedIn events
            - Internshala opportunities
            - Tech meetups and conferences
            - Hackathons
            - Workshops and training sessions
            - Industry networking events
            
            Gather at least 15-20 potential events with complete details.
            """,
            agent=self.search_analyst,
            expected_output="A comprehensive list of 15-20 events with full details"
        )
        
        task2 = Task(
            description="""
            Review the list of events and filter them based on:
            1. Relevance to the user's skills and interests
            2. Career advancement potential
            3. Skills development opportunities
            4. Networking value
            5. Credibility of the organizing entity
            6. Timing and accessibility
            
            Narrow down to 8-10 most promising events.
            """,
            agent=self.event_curator,
            expected_output="A filtered list of 8-10 high-quality events with analysis of each"
        )
        
        task3 = Task(
            description="""
            Create the final recommendation of exactly 5 events. For each event, provide:
            1. Event title and brief description
            2. Why it's specifically relevant to this user (reference their skills/goals)
            3. Key skills or knowledge they will gain
            4. Date, time, and format (online/offline)
            5. How to register or apply
            6. Direct link to the event
            
            Format the recommendations in a clear, structured way that highlights the 
            personalized nature of each suggestion.
            """,
            agent=self.recommendation_expert,
            expected_output="Five personalized event recommendations with detailed explanation of relevance"
        )
        
        return [task1, task2, task3]
    
    def search_web(self, query, num_results=10):
        """Search the web using Serper API"""
        headers = {
            'X-API-KEY': SERPER_API_KEY,
            'Content-Type': 'application/json'
        }
        
        payload = {
            "q": query,
            "num": num_results
        }
        
        response = requests.post(
            'https://google.serper.dev/search',
            headers=headers,
            json=payload
        )
        
        if response.status_code == 200:
            return response.json()
        else:
            return f"Error: {response.status_code} - {response.text}"
    
    def generate_event_recommendations(self, profile):
        """Run the crew to generate event recommendations"""
        user_profile = profile
        profile_summary = self._format_user_profile(user_profile)
        
        # Set the result as the crew's kickoff input
        result = self.crew.kickoff(
            inputs={"user_profile": profile_summary}
        )
        
        return result
        
    def _format_user_profile(self, profile):
        """Format the user profile for easier processing by agents"""
        formatted = "USER PROFILE SUMMARY:\n"
        formatted += f"Education: {profile.get('education', 'Not specified')}\n"
        formatted += f"Skills: {', '.join(profile.get('skills', []))}\n"
        formatted += f"Current Status: {profile.get('current_status', 'Not specified')}\n"
        formatted += f"Experience: {profile.get('experience_years', 'Not specified')} years\n"
        
        if 'last_job' in profile and profile['last_job']:
            formatted += f"Last Job: {profile['last_job'].get('title', '')} at {profile['last_job'].get('company', '')}\n"
        
        if 'job_preferences' in profile and profile['job_preferences']:
            job_prefs = profile['job_preferences']
            formatted += f"Job Type Preference: {job_prefs.get('type', 'Not specified')}\n"
            formatted += f"Preferred Roles: {', '.join(job_prefs.get('roles', []))}\n"
            formatted += f"Short-term Goal: {job_prefs.get('short_term_goal', 'Not specified')}\n"
            formatted += f"Long-term Goal: {job_prefs.get('long_term_goal', 'Not specified')}\n"
        
        if 'location' in profile and profile['location']:
            location = profile['location']
            formatted += f"Location: {location.get('city', 'Not specified')}\n"
            formatted += f"Willing to Relocate: {'Yes' if location.get('relocation', False) else 'No'}\n"
            formatted += f"Work Mode Preference: {location.get('work_mode', 'Not specified')}\n"
        
        if 'community' in profile and profile['community']:
            community = profile['community']
            formatted += f"Wants Mentorship: {'Yes' if community.get('wants_mentorship', False) else 'No'}\n"
            formatted += f"Mentorship Type: {community.get('mentorship_type', 'Not specified')}\n"
            formatted += f"Interested in Events: {'Yes' if community.get('join_events', False) else 'No'}\n"
        
        return formatted

# Example usage
if __name__ == "__main__":
    # Example user profile from the request
    sample_profile = {
        "user_id": "6809c002a03a7a1e240ab91e",
        "education": "Bachelor's Degree",
        "skills": ["Python", "Java", "AI", "NLP", "ML", "DL", "web development", "app development"],
        "current_status": "Looking for Work",
        "experience_years": 1,
        "last_job": {"title": "AI developer", "company": "OLVT"},
        "life_stage": {"pregnancy_status": "No", "needs_flexible_work": False, "situation": "None of the above"},
        "job_preferences": {
            "type": "Remote Work",
            "roles": ["Software"],
            "short_term_goal": "Upskill and crack good placement",
            "long_term_goal": "Yes, i want to be an entrepreneur"
        },
        "location": {"city": "Tirupati", "relocation": True, "work_mode": "Flexible"},
        "community": {"wants_mentorship": True, "mentorship_type": "Skill development", "join_events": True},
        "communication_preference": "Email",
        "consent": True
    }
    
    # Initialize the system with the sample profile
    recommender = EventRecommenderSystem(user_profile=sample_profile)
    
    # Generate recommendations
    recommendations = recommender.generate_event_recommendations(sample_profile)
    
    # Print the results
    print(recommendations)

ValidationError: 1 validation error for Agent
tools.0
  Input should be a valid dictionary or instance of BaseTool [type=model_type, input_value=<bound method EventRecomm... at 0x000002858C03DB50>>, input_type=method]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type